In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [87]:
import cudf as pd
import cupy as cp
import cuml
from cuml import LinearRegression
from cuml.linear_model import LinearRegression
from cuml import Ridge
from cuml.linear_model import Ridge
from cuml.model_selection import train_test_split
from cuml.linear_model import Lasso
from cuml.ensemble import RandomForestRegressor

from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import TimeSeriesSplit

from sklearn import neighbors
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

import numpy as np
import pandas as pd2 
import matplotlib.pylab as plt
import math

#from sklearn.linear_model import LinearRegression

# from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import plot_model

In [88]:
data = '/kaggle/input/110-1-ntut-dl-app-hw3/IXIC.csv'
df = pd.read_csv(data)
df

In [89]:
df.info()

In [90]:
# number of rows and columns in the data set
df.shape

In [91]:
print("Null Value Present: ", df.isnull().values.any())

In [92]:
df.isnull().sum

In [93]:
# Visualize the closing price history

data_plot = pd2.read_csv(data, sep=',', parse_dates=['Date'], index_col='Date')

plt.figure(figsize=(16,8))
plt.plot(data_plot['Close'])
plt.xlabel('Dates', fontsize=18)
plt.ylabel('Closing Prices', fontsize=18)
plt.show()

In [94]:
# Creating a new dataframe with only the 'Close' column
df2 = pd2.read_csv(data)
new_df = df2.filter(['Close'])
# Converting the df to a numpy array 
dataset = new_df.values
# Divide the dataset such that 80% of rows go under training and rest 20% will be used for testing
training_data_len = math.ceil( len(dataset) * 0.8)

training_data_len

In [11]:
# Scale the data (Feature Scaling)
scaler = MinMaxScaler(feature_range=(0,1))
# transforming our data 
scaled_data = scaler.fit_transform(dataset)

scaled_data

# LSTM 

In [29]:
# LSTMs expect our data to be in a specific format, usually a 3D array. 
# We start by creating data in 60 timesteps and converting it into an array using NumPy. 
# Next, we convert the data into a 3D dimension array with X_train samples, 60 timestamps, and one feature at each step.

# Creating the scaled training dataset 
train_data = scaled_data[0:training_data_len , :]

#splitting the data into X_train and y_train data sets 
X_train = []
Y_train = []

for i in range(60, len(train_data)):
    X_train.append(train_data[i-60:i, 0])
    Y_train.append(train_data[i,0])
    if i<=61:
        print(X_train)
        print(Y_train)
        print()


In [30]:
#convert the x_train and y_train to numpy array 
X_train, Y_train = np.array(X_train), np.array(Y_train)

In [31]:
#Reshape the data 
#Our X_train is 2 dimensional , but LSTM model wants 3 dimensional
print(X_train.shape)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

X_train.shape

In [32]:
#Build the LSTM model 
# We add the LSTM layer and later add a few Dropout layers to prevent overfitting. 
# We add the LSTM layer with the following arguments:
# 1. 50 units which is the dimensionality of the output space
# 2. return_sequences=True which determines whether to return the last output in the output sequence, or the full sequence
# 3. input_shape as the shape of our training set.

# When defining the Dropout layers, we specify 0.2, meaning that 20% of the layers will be dropped.
# Thereafter, we add the Dense layer that specifies the output of 1 unit
# After this, we compile our model using the popular adam optimizer and set the loss as the mean_squarred_error.
# Next, we fit the model to run on 100 epochs with a batch size of 32

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))

model.add(LSTM(50, return_sequences=False))

model.add(Dense(25))

model.add(Dense(1))

In [33]:
plot_model(model, show_shapes=True, show_layer_names=True)

In [34]:
#Compile the model 
model.compile(optimizer='adam', loss='mean_squared_error')

In [35]:
#Train the model
model.fit(X_train, Y_train, batch_size=64, epochs=10, verbose=1)

In [36]:
#Creating the scaled testing dataset from the remaining 20% 
test_data = scaled_data[training_data_len-60: , :]
#Create X_test and Y_test
X_test = []
Y_test = scaled_data[training_data_len: , :]

for i in range(60, len(test_data)):
    X_test.append(test_data[i-60:i, 0])

In [37]:
#Convert to numpy array 
X_test = np.array(X_test)

In [38]:
#Reshape the data 
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [39]:
X_test.shape

In [40]:
#Get the model's predicted price values 
predictions = model.predict(X_test)

In [41]:
#Evaluating the model using RMSE
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(Y_test, predictions)
rmse = math.sqrt(MSE)

In [42]:
rmse

In [45]:
# Plot the data 
train = df2[: training_data_len]
valid = df2[training_data_len :]

#unscaling the values back to the orignal form
predictions = scaler.inverse_transform(predictions)

valid['Predictions'] = predictions

#Visualize the data 
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Closing Price', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Value', 'Predictions'], loc = 'lower right')
plt.show()


In [46]:
#Show the valid and predicted prices
valid

In [48]:
# predicting the final closing price
data_2 = df2

new_df_2 = data_2.filter(['Close'])

temp = new_df_2[-60:].values
temp_scaled = scaler.transform(temp)
X_test_2 = []
X_test_2.append(temp_scaled)
X_test_2 = np.array(X_test_2)
X_test_2 = np.reshape(X_test_2, (X_test_2.shape[0], X_test_2.shape[1], 1))
final_y_predict = model.predict(X_test_2)
final_y_predict = scaler.inverse_transform(final_y_predict)
print(final_y_predict)

# Linear Regression 

In [12]:
#Set Target Variable
output_var = pd2.DataFrame(df2['Adj Close'])
#Selecting the Features
features = ['Open', 'High', 'Low', 'Volume']

In [25]:
#Scaling
scaler = MinMaxScaler()
feature_transform = scaler.fit_transform(df2[features])
output_var_scaled = scaler.fit_transform(output_var)
feature_transform= pd2.DataFrame(columns=features, data=feature_transform, index=df2.index)
feature_transform.head()

In [26]:
output_var_scaled

In [51]:
X_train = feature_transform[: training_data_len]
X_test = feature_transform[training_data_len : ]

Y_train = output_var_scaled[: training_data_len]
Y_test = output_var_scaled[training_data_len : ]

In [28]:
#All linear Regression Algorithms
algorithm = ['svd', 'eig', 'svd-qr', 'svd-jacobi']

#Evaluation Metrics for all 5 algorithms
for i in algorithm:
    print("Algorithm:")
    print(i)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    reg = lr.fit(X_train,Y_train)
    preds = lr.predict(X_test)
    print("RMSE:")
    print(np.sqrt(cuml.metrics.regression.mean_squared_error(Y_test.astype('int64'),preds.astype('int64'))))

# Ridge Regression

In [52]:
alpha = cp.array([1e-5])
ridge1 = Ridge(alpha = alpha, fit_intercept = True, normalize = False,
solver = "eig")
model1=ridge1.fit(X_train,Y_train)
y_pred_ridge1=model1.predict(X_test)

print(np.sqrt(cuml.metrics.regression.mean_squared_error(Y_test.astype('int64'),y_pred_ridge1.astype('int64'))))

In [53]:
alpha = cp.array([1e-5])
ridge2 = Ridge(alpha = alpha, fit_intercept = True, normalize = False,
solver = "svd")
model2=ridge2.fit(X_train,Y_train)
y_pred_ridge2=model2.predict(X_test)

print(np.sqrt(cuml.metrics.regression.mean_squared_error(Y_test.astype('int64'),y_pred_ridge2.astype('int64'))))

# Lasso Regression

In [54]:
ls = Lasso(alpha = 0.1)
result_lasso = ls.fit(X_train, Y_train)
y_pred_lasso=result_lasso.predict(X_test)

print(np.sqrt(cuml.metrics.regression.mean_squared_error(Y_test.astype('int64'),y_pred_lasso.astype('int64'))))

# k-Nearest Neighbours

In [65]:
# Taking the same training and testing data 
X_train = feature_transform[: training_data_len]
X_test = feature_transform[training_data_len : ]

Y_train = output_var_scaled[: training_data_len]
Y_test = output_var_scaled[training_data_len : ]

In [66]:
#using gridsearch to find the best parameter
params = {'n_neighbors':[2,3,4,5,6,7,8,9]}
knn = neighbors.KNeighborsRegressor()
model = GridSearchCV(knn, params, cv=5)

In [67]:
#fit the model and make predictions
model.fit(X_train,Y_train)
predictions = model.predict(X_test)

In [68]:
#rmse
rmse = np.sqrt(np.mean(np.power((np.array(Y_test)-np.array(predictions)),2)))
rmse

In [184]:
valid.head()

# Random Forest Regression Model

In [82]:
# Taking the same training and testing data 
X_train = feature_transform[: training_data_len]
X_test = feature_transform[training_data_len : ]

Y_train = output_var_scaled[: training_data_len]
Y_test = output_var_scaled[training_data_len : ]

In [83]:
forest_regressor = RandomForestRegressor(n_estimators = 250)
forest_regressor.fit(X_train.values.astype('float32'), Y_train.astype('float32'))

y_pred_random= forest_regressor.predict(X_test)

print("Root Mean Squared Error:", round(np.sqrt(metrics.mean_squared_error(Y_test, y_pred_random)), 4))